In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras.utils as image
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from keras import models
from keras import layers
from keras import optimizers
from PIL import Image
from sklearn.model_selection import train_test_split
from statistics import mean
from keras.applications import VGG16

In [3]:
def arrange_data(df):
    image_data = []
    img_paths = np.asarray(df.iloc[:, 0])
    
    for i in tqdm(range(len(img_paths))):
        img = image.load_img(img_paths[i],target_size=(200, 150, 3))
        img = image.img_to_array(img)
        img = img/255
        image_data.append(img)   
        
    X = np.array(image_data)
    Y = np.array(df.iloc[:,1:29])
    
    print("Shape of images:", X.shape)
    print("Shape of labels:", Y.shape)
    
    return X, Y

In [ ]:
print("Processing train..")
train_df = pd.read_csv("CSV/Train.csv", delimiter=" ")
X_train, Y_train = arrange_data (train_df)

print("Processing valid..")
val_df = pd.read_csv("CSV/Valid.csv", delimiter=" ")
X_val, Y_val = arrange_data (val_df)

In [ ]:
from keras.applications import VGG16

num_classes = 28 

vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(200, 150, 3))

for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

model = models.Sequential()
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='sigmoid'))

model.summary()

In [ ]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

with tf.device('/device:GPU:0'):
    model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,horizontal_flip=True, fill_mode="nearest")
    EPOCHS = 50
    BS = 64
    history = model.fit(aug.flow(X_train, Y_train, batch_size=BS), validation_data=(X_val, Y_val), steps_per_epoch=len(X_train) // BS, epochs=EPOCHS)
    model.save('Model_VGG.h5')